### Warning:

We faced a wide array of bugs when attempting to utilize the script out of the box. There will be code cells that was used for testing out bugs and isolating them down. Please disregard them. We have attempted to remove as many as possible, but we may have missed a few. The script, however, should work properly now despite our initial troubles.

In [ ]:
from bs4 import BeautifulSoup
import datetime
import unicodedata
import requests
import pandas as pd
import numpy as np
from time import sleep
import math
from config import Config
import dateutil.relativedelta
import pandas_market_calendars as mcal
import os
import io
import re
from tqdm import tqdm
import gc
import ast

In [ ]:
import signal
class Timeout():
    class TimeoutException(Exception):
        pass

    def _timeout(signum, frame):
        raise Timeout.TimeoutException()

    def __init__(self, timeout=10):
        self.timeout = timeout
        signal.signal(signal.SIGALRM, Timeout._timeout)

    def __enter__(self):
        signal.alarm(self.timeout)

    def __exit__(self, exc_type, exc_value, traceback):
        signal.alarm(0)
        return exc_type is Timeout.TimeoutException

### 1 Define Functions and Classes

In [ ]:
from bs4 import BeautifulSoup
import datetime
import unicodedata
import requests
import pandas as pd
import numpy as np
from time import sleep
import re
import requests

class SEC_Extractor:
    def get_doc_links(cik,ticker):
        try:
            base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
            inputted_cik = cik
            payload = {
                "action" : "getcompany",
                "CIK" : inputted_cik,
                "type" : "8-K",
                "output":"xml",
                "dateb" : "20180401",
            }
            sec_response = requests.get(url=base_url,params=payload)
            soup = BeautifulSoup(sec_response.text,'lxml')
            url_list = soup.findAll('filinghref')
            html_list = []
            # Get html version of links
            for link in url_list:
                link = link.string
                if link.split(".")[len(link.split("."))-1] == 'htm':
                    txtlink = link + "l"
                    html_list.append(txtlink)

            doc_list = []
            doc_name_list = []
            # Get links for txt versions of files
            for k in range(len(html_list)):
                txt_doc = html_list[k].replace("-index.html",".txt")
                doc_name = txt_doc.split("/")[-1]
                doc_list.append(txt_doc)
                doc_name_list.append(doc_name)
                # Create dataframe of CIK, doc name, and txt link
            df = pd.DataFrame(
                {
                "cik" : [cik]*len(html_list),
                "ticker" : [ticker]*len(html_list),
                "txt_link" : doc_list,
                "doc_name": doc_name_list
                }
            )
        except requests.exceptions.ConnectionError:
                sleep(.1)
        return df

    # Extracts text and submission datetime from document link
    def extract_text(link):
        try:
            r = requests.get(link)
            print("Done Link")
            #Parse 8-K document
            filing = BeautifulSoup(r.content,"html5lib",from_encoding="ascii")
            #Extract datetime
            try:
                submission_dt = filing.find("acceptance-datetime").string[:14]
            except AttributeError:
                    # Flag docs with missing data as May 1 2018 10AM
                submission_dt = "20180501100000"
            print("Done Date")
            submission_dt = datetime.datetime.strptime(submission_dt,"%Y%m%d%H%M%S")
            #Extract HTML sections
            for section in filing.findAll("html"):
                try:
                    #Remove tables
                    for table in section("table"):
                        table.decompose()
                    #Convert to unicode
                    section = unicodedata.normalize("NFKD",section.text)
                    section = section.replace("\t"," ").replace("\n"," ").replace("/s"," ").replace("\'","'")            
                except AttributeError:
                    section = str(section.encode('utf-8'))
            filing = "".join((section))
        except requests.exceptions.ConnectionError:
                sleep(10)
        sleep(.1)
        print ("Done filing")

        return filing, submission_dt

    def extract_item_no(document):
        pattern = re.compile("Item+ +\d+[\:,\.]+\d+\d")
        item_list = re.findall(pattern,document)
        return item_list


In [ ]:
from bs4 import BeautifulSoup
import datetime
import unicodedata
import requests
import pandas as pd
import numpy as np
from time import sleep
import re
import requests

class SEC_Extractor:
    def get_doc_links(cik,ticker):
        try:
            base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
            inputted_cik = cik
            payload = {
                "action" : "getcompany",
                "CIK" : inputted_cik,
                "type" : "8-K",
                "output":"xml",
                "dateb" : "20180401",
            }
            sec_response = requests.get(url=base_url,params=payload)
            soup = BeautifulSoup(sec_response.text,'lxml')
            url_list = soup.findAll('filinghref')
            html_list = []
            # Get html version of links
            for link in url_list:
                link = link.string
                if link.split(".")[len(link.split("."))-1] == 'htm':
                    txtlink = link + "l"
                    html_list.append(txtlink)

            doc_list = []
            doc_name_list = []
            # Get links for txt versions of files
            for k in range(len(html_list)):
                txt_doc = html_list[k].replace("-index.html",".txt")
                doc_name = txt_doc.split("/")[-1]
                doc_list.append(txt_doc)
                doc_name_list.append(doc_name)
                # Create dataframe of CIK, doc name, and txt link
            df = pd.DataFrame(
                {
                "cik" : [cik]*len(html_list),
                "ticker" : [ticker]*len(html_list),
                "txt_link" : doc_list,
                "doc_name": doc_name_list
                }
            )
        except requests.exceptions.ConnectionError:
                sleep(.1)
        return df

    # Extracts text and submission datetime from document link
    def extract_text(link):
      check = 0
      with Timeout(20):
        try:
                r = requests.get(link)
                print("Done Link")
                #Parse 8-K document
                filing = BeautifulSoup(r.content,"html5lib",from_encoding="ascii")
                #Extract datetime
                try:
                    submission_dt = filing.find("acceptance-datetime").string[:14]
                except AttributeError:
                        # Flag docs with missing data as May 1 2018 10AM
                    submission_dt = "20180501100000"
                print("Done Date")
                submission_dt = datetime.datetime.strptime(submission_dt,"%Y%m%d%H%M%S")
                #Extract HTML sections
                for section in filing.findAll("html"):
                    try:
                        #Remove tables
                        for table in section("table"):
                            table.decompose()
                        #Convert to unicode
                        section = unicodedata.normalize("NFKD",section.text)
                        section = section.replace("\t"," ").replace("\n"," ").replace("/s"," ").replace("\'","'")            
                    except AttributeError:
                        section = str(section.encode('utf-8'))
                filing = "".join((section))
        except requests.exceptions.ConnectionError:
                sleep(10)
        sleep(.1)
        print ("Done filing")
        return filing, submission_dt
        check = 1
      print(check)
      if check == 0:
        return ("Passed", "Passed")
      
    def extract_item_no(document):
      if document != "Passed":
        pattern = re.compile("Item+ +\d+[\:,\.]+\d+\d")
        item_list = re.findall(pattern,document)
        return item_list
      else:
        return


## Note:

One might notice that we are using normal opening price and adjusted closing price for the index, when we are using adjusted opening and closing price for the stock prices. For the index, there is no difference between normal and adjusted price for both the GSPC and VIX. This may be likely to the fact that they may not have stock splits, dividends, and so forth to account for. So, this is not a problem.

So, we felt comfortable in using adjusted opening and adjusted closing price for the stock prices. It does not create conflict.

In [ ]:
# Returns Dataframe of document links for a given CIK
idx = pd.Index
class FinDataExtractor:
    def __init__(self):
        # S&P 500 index data downloaded from Yahoo Finance GSPC
        self.gspc_df = pd.read_csv("Data/Indexes/gspc.csv",parse_dates=['Date'],index_col="Date")
        # Get VIX index data downloaded from Yahoo Finance
        self.vix_df = pd.read_csv("Data/Indexes/vix.csv",parse_dates=['Date'],index_col="Date")
        nyse = mcal.get_calendar('NYSE')
        self.nyse_holidays = nyse.holidays().holidays
        self.all_tickers_data = pd.read_pickle("Pickles/all_tickers_data.pkl")
        
#Takes datetime object and ticker string, returns price (opening or closing)
    def get_historical_movements(self,row,period):
        ticker,release_date = row[0],row[1]
        print(ticker)
        print (release_date)
        print (type(release_date))
       #1 Week
        if period == "week":
            e_start = release_date + datetime.timedelta(weeks=-1)
            b_start = e_start

            e_end = release_date + dateutil.relativedelta.relativedelta(days=-1)
            b_end = e_end

         #1 Month    
        elif period == "month":
            e_start = release_date + dateutil.relativedelta.relativedelta(months=-1)
            b_start = e_start + dateutil.relativedelta.relativedelta(days=-5)

            e_end = release_date + dateutil.relativedelta.relativedelta(days=-1)
            b_end = release_date + dateutil.relativedelta.relativedelta(days=-6)

        #1 Quarter
        elif period == "quarter":
            e_start = release_date + dateutil.relativedelta.relativedelta(months=-3)
            b_start = e_start + dateutil.relativedelta.relativedelta(days=-10)

            e_end = release_date + dateutil.relativedelta.relativedelta(days=-1)
            b_end = release_date + dateutil.relativedelta.relativedelta(days=-11)

        #1 Year
        elif period == "year":
            e_start = release_date + dateutil.relativedelta.relativedelta(years=-1)
            b_start = e_start + dateutil.relativedelta.relativedelta(days=-20)

            e_end = release_date + dateutil.relativedelta.relativedelta(days=-1)
            b_end = release_date + dateutil.relativedelta.relativedelta(days=-21)
        else:
            print("error")
            raise KeyError
        print("test")
        e_start = self.weekday_check(e_start)
        b_start = self.weekday_check(b_start)
        e_end = self.weekday_check(e_end)
        b_end = self.weekday_check(b_end)

        start_price = self.get_av_data(ticker=ticker,start_date = b_start, end_date = e_start)
        end_price = self.get_av_data(ticker=ticker,start_date = b_end, end_date = e_end)
        stock_change = self.calculate_pct_change(end_price,start_price)
    
        start_index = self.get_index_price(start_date = b_start, end_date = e_start)
        end_index = self.get_index_price(start_date = e_start, end_date = e_end)
        index_change =  self.calculate_pct_change(end_index,start_index)

        print(start_price)
        print(end_price)
        print(stock_change)
        print(start_index)
        print(end_index)
        print(index_change)
        
        normalized = stock_change - index_change
        print (normalized)
        return normalized

    def DISABLEDget_av_data(self,ticker,start_date,end_date,market_open=False):
        start_date = start_date.date()
        end_date = end_date.date()
        print (start_Date)
        print (end_date)

        try:
            if market_open == False:
              '''
            Original developer had his dataset set up to slice from end date to start date
            which caused the stock price to be set to np.nan. It may have worked fine with Alpha Vantage
            somehow, but it does not for Wiki Data.
            '''
                price = self.all_tickers_data.xs(ticker,0).loc[end_date:start_date,"adjusted_close"].mean()
            else:
                price = self.all_tickers_data.xs(ticker,0).loc[end_date:start_date,"open"].mean()
        except (KeyError,IndexError):
            price = np.nan
        print ("price", price)
        return price
      
    def get_av_data(self, ticker,start_date,end_date,market_open=False):
        start_date = start_date.date()
        end_date = end_date.date()

        try:
            if market_open == False:
              price = wikidf2.xs(ticker,0).set_index('date').loc[start_date:end_date, "adj_close"].mean()
                # price = self.all_tickers_data.xs(ticker,0).loc[end_date:start_date,"adjusted_close"].mean()
            else:
              price = wikidf2.xs(ticker,0).set_index('date').loc[start_date:end_date, "adj_open"].mean()
                # price = self.all_tickers_data.xs(ticker,0).loc[end_date:start_date,"adj_open"].mean()
        except (KeyError,IndexError):
            price = np.nan
        return price
            
      

    # Takes ticker, 8K release date, checks time of release and then calculate before and after price change
    def get_change(self,row):
        release_date = row['release_date']
        print (release_date)
        ticker = row['ticker']
        print (ticker)
        market_close = release_date.replace(hour=16,minute=0,second=0)
        market_open = release_date.replace(hour=9,minute=30,second=0)

    # If report is released after market hours, take change of start date close and release date open
        if release_date > market_close:
            start_date = release_date
            end_date = release_date + datetime.timedelta(days=1)
            end_date = self.weekday_check(end_date)
            end_date = end_date

            price_before_release = self.get_av_data(ticker,start_date,start_date,market_open=False)
            price_after_release = self.get_av_data(ticker,end_date,end_date,market_open=True)

            index_before_release = self.get_index_price(start_date,start_date,market_open=False)
            index_after_release = self.get_index_price(end_date,end_date,market_open=True)

            try:
                vix = self.vix_df.loc[self.vix_df.index == np.datetime64(start_date.date()),"Adj Close"][0].item()
            except IndexError:
                vix = np.nan

        # If report is released before market hours, take change of start date's close and release date's open
        elif release_date < market_open:
            start_date = release_date + datetime.timedelta(days=-1)
            start_date = self.weekday_check(start_date)
            end_date = release_date

            price_before_release = self.get_av_data(ticker,start_date,start_date,market_open=False)
            price_after_release = self.get_av_data(ticker,end_date,end_date,market_open=True) 

            index_before_release = self.get_index_price(start_date,start_date,market_open=False)
            index_after_release = self.get_index_price(end_date,end_date,market_open=True)
            try:
                vix = self.vix_df.loc[self.vix_df.index == np.datetime64(start_date.date()),"Adj Close"][0].item()
            except IndexError:
                vix = np.nan
        # If report is released during market hours, use market close
        else:
            start_date = release_date
            end_date = release_date
            price_before_release = self.get_av_data(ticker,start_date,start_date,market_open=True)
            price_after_release = self.get_av_data(ticker,end_date,end_date,market_open=False)

            index_before_release = self.get_index_price(start_date,start_date,market_open=True)
            index_after_release = self.get_index_price(end_date,end_date,market_open=False)
            
            try:
                vix = self.vix_df.loc[self.vix_df.index == np.datetime64(start_date.date()),"Open"][0].item()
            except IndexError:
                vix = np.nan
        print(price_after_release)
        print(price_before_release)
        price_pct_change = self.calculate_pct_change(price_after_release,price_before_release)
        index_pct_change = self.calculate_pct_change(index_after_release,index_before_release)
        normalized_change = price_pct_change - index_pct_change
        print (normalized_change)
        print (vix)
        return normalized_change, vix

    def get_index_price(self,start_date,end_date,market_open=False):
        try:
            if market_open == True:
                price = self.gspc_df.loc[(self.gspc_df.index >= np.datetime64(start_date.date())) & 
                                 (self.gspc_df.index <= np.datetime64(end_date)),"Open"].mean()
            else:
                price = self.gspc_df.loc[(self.gspc_df.index >= np.datetime64(start_date.date())) & 
                                 (self.gspc_df.index <= np.datetime64(end_date)),"Adj Close"].mean()
        except IndexError:
                price = np.nan
        return price

    def calculate_pct_change(self,end_value,start_value):
        pct_change = (end_value - start_value) / start_value
        pct_change = round(pct_change,4) * 100
        return pct_change

    def weekday_check(self,date):  
        while date.isoweekday() > 5 or date.date() in self.nyse_holidays:
            date = date + datetime.timedelta(days=-1)
        return date

### 2. Get S&P 500 Data

In [ ]:
# Get table of the S&P 500 tickers, CIK, and industry from Wikipedia
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
cik_df = pd.read_html(wiki_url,header=0,index_col=0)[0]
cik_df['GICS Sector'] = cik_df['GICS Sector'].astype("category")
cik_df['GICS Sub Industry'] = cik_df['GICS Sector'].astype("category")
cik_df.head()

### 3. Get List of 8K links from SEC Edgar

In [ ]:
sec_ext = SEC_Extractor
no_parts = 2
part_no = 3

In [ ]:
df_list = []
company_list = cik_df['CIK'].to_dict()
for (ticker,cik) in tqdm(company_list.items()):
    df_list.append(sec_ext.get_doc_links(cik,ticker))
doc_links_df = pd.concat(df_list,axis=0)
doc_links_df = doc_links_df.set_index("ticker").join(cik_df['GICS Sector']).join(cik_df['GICS Sub Industry']).reset_index().rename(columns={"index":"ticker"})
doc_links_df.head()


In [ ]:
doc_links_df.to_csv("doc_links_df.csv")
doc_links_df.to_pickle("Pickles/doc_links_df.pkl")

### 4. Download 8Ks & Stock Movements

In [ ]:
crawled_df = pd.read_pickle("Pickles/doc_links_df.pkl")

In [ ]:
start = len(pd.read_csv("Data/texts_example1.csv"))

In [ ]:
crawled_df = crawled_df.iloc[start:, :] # Skip 842. We also miss out on 840 and 841

In [ ]:
crawled_df.head()

In [ ]:
while part_no > no_parts:
    no_parts = int(input("Split data into how many parts?"))
    part_no = int(input("Which part is this?"))
    
chunksize = int(input("Number of rows to process at once (10 to 50 recommended)"))

#Load pickle
# crawled_df = np.array_split(pd.read_pickle("Pickles/doc_links_df.pkl"),no_parts)[part_no-1]
crawled_df =  np.array_split(crawled_df, no_parts)[part_no-1]
crawled_len = len(crawled_df['txt_link'])


df_list = []
for i, df in tqdm(enumerate(np.array_split(crawled_df,chunks))):
    print(i)
    df['text'], df['release_date'] = zip(*df['txt_link'].apply(sec_ext.extract_text))
    # df['text'], df['release_date'] = sec_ext.extract_text(df['txt_link'])
    
    df['items'] = df['text'].map(sec_ext.extract_item_no)
    if not os.path.isfile("Data/texts_example{}.csv".format(part_no)): #If no file exists, create one with header
        df.to_csv("Data/texts_example{}.csv".format(part_no),chunksize=chunksize,)
    else: # else it exists so append without writing the header
        df.to_csv("Data/texts_example{}.csv".format(part_no),mode="a",header=False,chunksize=chunksize)       
    df_list.append(df)
    del df
    
    if i % 50 == 0:
        gc.collect()
df = pd.concat(df_list)
df.head()


In [ ]:
df.head()

In [ ]:
crawled_df = pd.read_pickle("Pickles/doc_links_df.pkl")
crawled_len = len(crawled_df['txt_link'])



### Load Financial Data

In [ ]:
import copy

In [ ]:
df = pd.read_csv("Data/texts_example1.csv")

In [ ]:
df3 = copy.deepcopy(df)

In [ ]:
df3.head()

In [ ]:
cik_dict = cik_df['Symbol'].to_dict()
cik_dict = {v: k for v, k in cik_dict.items()}
df3['ticker'] = df3['ticker'].map(cik_dict)

In [ ]:
wikidf = pd.read_csv('WIKI_PRICES_212b326a081eacca455e13140d7bb9db.csv')

In [ ]:
wikidf.head()

In [ ]:
wikidf['date']  = pd.to_datetime(wikidf['date'])

In [ ]:
wikidf[(wikidf["ticker"] == "MMM")].iloc[-10:,:]

In [ ]:
wikidf[(wikidf["date"] == df['release_date'][0].normalize())].iloc[-10:,:]


In [ ]:
wikidf2 = wikidf.set_index('ticker')

In [ ]:
wikidf2.xs('MMM',0).iloc[-200:, :]

In [ ]:
type(wikidf2.iloc[1, 0])

In [ ]:
wikidf2.xs('MMM',0).set_index('date').loc[pd.to_datetime('1970-01-02'):pd.to_datetime('1970-01-06'), "adj_open"].mean()

In [ ]:
wikidf2.xs('MMM',0).set_index('date').loc[pd.to_datetime('2018-02-20'), "adj_open"].mean() # Missing Data

In [ ]:
try:
  wikidf[(wikidf["ticker"] == "MMM" and wikidf["date"] == df['release_date'][0].normalize())] 
except ValueError:
  wikidf[(wikidf["ticker"] == "MMM" and wikidf["date"] == wikidf['release_date'][-1].normalize())] 

In [ ]:
df = copy.deepcopy(df3)

In [ ]:
# Mask
df2 = df[(df['text'] != "Passed")]
df2 = df2[(df2['release_date'] != "Passed")]

In [ ]:
df2 = df2.reset_index(drop=True)

In [ ]:
df2['release_date']  = pd.to_datetime(df2['release_date'] )

In [ ]:
# Find rows flagged where no date was found
df2 = df2.loc[~(df2['release_date'] >= pd.datetime(year=2018,month=5,day=1))]
df2 = df2.drop_duplicates(subset="doc_name")
df2.index.names = ['ticker']
# df = df.reset_index()

In [ ]:
len(df2)

In [ ]:
df = copy.deepcopy(df2)

In [ ]:
type(df['release_date'][0])

In [ ]:
#from FinDataExtractor import FinDataExtractor
fin_data = FinDataExtractor()
## Load pickle of ticker, date, and doc number


tempContainer = df[['ticker','release_date']].apply(fin_data.get_change,axis=1)
tempContainer.rename({"ticker": "price_change", "release_date": "VIX"}, axis = 1, inplace = True)
df = df.join(tempContainer)
print ("pc done", df.head())
df['rm_week'] = df[['ticker','release_date']].apply(fin_data.get_historical_movements,period="week",axis=1)
print ("week done", df.head())
df['rm_month'] = df[['ticker','release_date']].apply(fin_data.get_historical_movements,period="month",axis=1)
print ("month done", df.head())
df['rm_qtr'] = df[['ticker','release_date']].apply(fin_data.get_historical_movements,period="quarter",axis=1)
print ("quart done", df.head())
df['rm_year'] = df[['ticker','release_date']].apply(fin_data.get_historical_movements,period="year",axis=1)
print ("year done", df.head())
df["signal"] = df['price_change'].map(lambda x: np.nan if math.isnan(x) else ("stay" if -1<x<1  else ("up" if x>1 else "down")))
df.head()